In [1]:
import datasets
from os.path import join
import tensorflow as tf
import torch

In [3]:

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
params_files = []

In [4]:
bert_model_name= 'google/bert_uncased_L-2_H-128_A-2'

In [5]:
import sys
sys.path.append("../")
from data.manual_labels.data_reader import ManualLabels

In [6]:
from features_processing.bert_tokenizer import BERTTokenizer

Using TensorFlow backend.


## DATA

In [7]:
data =[{'id':'any_cancer_IMPRESS','type': 'manual_label' , 'params': {'outcome': 'any_cancer', 'text': 'IMPRESS'}}]
data = ManualLabels(**data[0]['params'])
x_train, x_validate, x_test, y_train, y_validate, y_test, info_train, info_validate, info_test, columns = data.get_train_validate_test()

(2570, 16) (2570,)


In [8]:
tokenizer= BERTTokenizer(bert_model_name, True, True, 512)

## Tokenizer

In [9]:
x_train_encoded = tokenizer.transform(list(x_train))
x_val_encoded = tokenizer.transform(list(x_validate))

In [35]:
x_train_encoded[0]

Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [10]:
type(x_train_encoded)

transformers.tokenization_utils_base.BatchEncoding

In [11]:
x_train_encoded[2]

Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [12]:
len(list(x_train))

2043

In [13]:
x_train.shape

(2043,)

In [14]:
class TorchDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [15]:
def get_dataset(encodings,labels, type='tf'):
    if type=='tf':
        dic= (encodings,labels) #not tested
        return tf.data.Dataset.from_tensor_slices(dic)
    else:
        return TorchDataset(encodings, labels)

In [48]:
x_val_encoded[0]

Encoding(num_tokens=321, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [16]:
train_dataset = get_dataset(x_train_encoded,y_train, type='torch')
val_dataset = get_dataset(x_val_encoded,y_validate, type='torch')

In [50]:
val_dataset[0]['input_ids']

tensor([  101,  8605,  1024, 11192,  4456,  1010,  2717, 16594, 13594,  1012,
         7263, 13483,  3623,  1999,  2157, 11192,  7293, 16308,  1012,  2053,
        15298,  2047,  2609,  1997,  4295,  1012,  1045,  1010,  1996,  4252,
         7522,  1010,  2031,  8182,  1996,  4871,  1998,  5993,  2007,  1996,
         3189,  2004,  2517,  1012,   102,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0, 

## Model

In [18]:
bert_mini = 'google/bert_uncased_L-4_H-256_A-4'
model_name = bert_mini

## Training

In [19]:
from transformers import glue_convert_examples_to_features, BertTokenizer, AutoTokenizer
from transformers import TFAutoModelForSequenceClassification, AutoModelForSequenceClassification, TFAutoModel, AutoModel

In [20]:
# from transformers import TFDistilBertForSequenceClassification, TFTrainer, TFTrainingArguments
from transformers import  TFTrainer, TFTrainingArguments, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

# with training_args.strategy.scope():
model = AutoModelForSequenceClassification.from_pretrained(model_name)
#     model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)


Some weights of the model checkpoint at google/bert_uncased_L-4_H-256_A-4 were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification w

In [21]:
for name, param in model.named_parameters():
#     print (name)
    if 'classifier' not in name: # classifier layer
        param.requires_grad = False

In [22]:
trainer.train()

Step,Training Loss
10,0.671025
20,0.684205
30,0.670889
40,0.703035
50,0.693403
60,0.663061
70,0.672130
80,0.667905
90,0.709429
100,0.712643


TrainOutput(global_step=384, training_loss=0.6869475816686949)

In [24]:
train_prediciton = trainer.predict(train_dataset)

In [26]:
train_prediciton.predictions.shape

(2043, 2)

In [27]:
val_prediciton = trainer.predict(val_dataset)
val_prediciton.predictions

(228, 2)

In [30]:
val_prediciton.predictions.argmax(axis=1).shape

(228,)

In [31]:
x_validate.shape

(228,)

In [32]:
val_prediciton.predictions

array([[-0.36307108, -0.06306561],
       [-0.39728653, -0.04404884],
       [-0.44236404, -0.07473429],
       [-0.4295658 , -0.07578716],
       [-0.2708267 , -0.09292857],
       [-0.37386972, -0.07288186],
       [-0.3983547 , -0.04792517],
       [-0.39643368, -0.04444391],
       [-0.43347958, -0.04299917],
       [-0.3757126 , -0.10583872],
       [-0.39969778, -0.09234884],
       [-0.37930778, -0.03551338],
       [-0.35765976, -0.09728746],
       [-0.42904067, -0.08488195],
       [-0.38601953, -0.06436409],
       [-0.34401393, -0.03371797],
       [-0.46314323, -0.12679453],
       [-0.36545756, -0.11137249],
       [-0.26646876, -0.12252057],
       [-0.25250682, -0.03125205],
       [-0.21499565, -0.07627793],
       [-0.2615397 , -0.041428  ],
       [-0.4275329 , -0.09229673],
       [-0.34965083, -0.03099173],
       [-0.07864101, -0.04274325],
       [-0.36078   , -0.04968891],
       [-0.45129833, -0.04903188],
       [-0.37375835, -0.03004476],
       [-0.38327333,